In [104]:
import pandas as pd
import numpy as np
import os
import subprocess
import shutil
import generator
import json
from pathlib import Path
import sklearn.cluster
from sklearn.metrics import f1_score

In [106]:
DB, labels = generator.generate_subspacedata(100, 100, False, [[5, 4, 1, 0.001],
                                                              [5, 6, 1, 0.001],
                                                              [5, 9, 1, 0.001],
                                                              [5, 9, 1, 0.001],
                                                              [5, 9, 1, 0.001]])
np.savetxt("res/sample6.csv", DB, delimiter=",", fmt='%1.8f')

# generates the corresponding ground_truth.csv file from the labels matrix
with open("res/ground_truth.csv", mode="w") as output_file:
    cluster_counter = 1
    while np.argwhere(labels == cluster_counter).size > 0:
        dimensions = list(set(np.argwhere(labels == cluster_counter)[:,1]))
        U = list(set(np.argwhere(labels == cluster_counter)[:,0]))
        cluster_counter += 1
        output_file.write(str(dimensions) + "-" + str(U) + "\n")

In [107]:
dirpath = Path('out')
if dirpath.exists() and dirpath.is_dir():
    shutil.rmtree(dirpath)

SAMPLEFILE = '6'
GERMAN_FILE_FORMATTING = 'true'
EPSILON = '0.5'
MINPTS = '4'
SPLITTING_SIZE = '16'
EVENLY_SIZED_SLICES = 'true'
DBSCAN = 'false'
p = subprocess.Popen(['java',
                      '-jar',
                      'SubScaleExtended.jar',
                      SAMPLEFILE,
                      GERMAN_FILE_FORMATTING,
                      EPSILON,
                      MINPTS,
                      SPLITTING_SIZE,
                      EVENLY_SIZED_SLICES,
                      DBSCAN],
                     stdout=subprocess.PIPE,
                     stderr=subprocess.STDOUT)
for line in p.stdout:
    print(str(line, "utf-8").replace('\r\n', ''))

Duration for creating the database: 38ms
Duration for calculating the density chunks: 0ms
Slice number: 8, size of collision map: 27
Slice number: 0, size of collision map: 1
Duration for calculating subspaces in slice 0: 0ms
Slice number: 1, size of collision map: 4
Duration for calculating subspaces in slice 1: 1ms
Slice number: 2, size of collision map: 9
Duration for calculating subspaces in slice 2: 0ms
Slice number: 3, size of collision map: 12
Duration for calculating subspaces in slice 3: 0ms
Slice number: 4, size of collision map: 23
Duration for calculating subspaces in slice 4: 0ms
Slice number: 5, size of collision map: 30
Duration for calculating subspaces in slice 5: 0ms
Slice number: 6, size of collision map: 20
Duration for calculating subspaces in slice 6: 0ms
Slice number: 7, size of collision map: 11
Duration for calculating subspaces in slice 7: 0ms
Slice number: 8, size of collision map: 17
Duration for calculating subspaces in slice 8: 0ms
Slice number: 9, size of

In [108]:
DB

array([[-29.93002545,  22.56522383,  61.69234081, ..., -43.29171744,
        -10.21352848,  42.79073344],
       [-29.93068444,  22.56529513,  61.69310692, ...,  77.10241646,
        -69.79696597,  48.81951259],
       [-29.93005704,  22.56530251,  61.6933578 , ...,   7.81001096,
         -6.44370418, -86.12515931],
       ...,
       [-96.38482979,  44.49869041,   7.50145534, ..., -99.19835352,
        -46.45175958,  98.21611606],
       [-87.19415821,  45.08820912,  27.55515226, ..., -80.66122312,
         89.98719628,  25.92143484],
       [ 67.84659032, -46.12026085,  15.20744954, ...,  39.30186145,
        -83.41124807,  31.83975919]])

In [133]:
# Potential subspaces will be searched for clusters using the dbscan algorithm

with open("out/results/found_clusters.csv", mode="w") as output_file:
    with os.scandir("out/results/subspaces") as it:
        for entry in it:
            if os.path.getsize(entry.path) > 0: # Check for empty file sometimes generated due to bug in SubScaleExtended.jar
                df = pd.read_csv(entry.path, header=None, delimiter="-")
                for _, row in df.iterrows():
                    points_indexes = json.loads(row[1])
                    dimensions = json.loads(row[0])
                    S = DB[points_indexes][:,dimensions]
                    clustering = sklearn.cluster.DBSCAN(eps=0.004).fit(S)

                    # Converting dbscan result in clusters.csv file-format
                    unique_labels = set(clustering.labels_)
                    unique_labels.discard(-1) # -1 stands for noisy samples and is therefore removed
                    for k in unique_labels:
                        U = np.array(points_indexes)[clustering.labels_ == k]
                        output_file.write(str(dimensions) + "-" + str(U.tolist()) + "\n")

In [134]:
# f1-measure
with open("out/results/found_clusters.csv", mode="r") as found_clusters_file, \
     open("res/ground_truth.csv", mode="r") as ground_truth_file:

    found_clusters_df = pd.read_csv(found_clusters_file, header=None, delimiter="-")
    ground_truth_df = pd.read_csv(ground_truth_file, header=None, delimiter="-")


In [ ]:
# https://www.ipd.kit.edu/~muellere/publications/BTW2011.pdf Erklärung zu f1 measure
labels